In [1]:
import numpy as np


class SVM:

    def __init__(self, C = 1.0):
        # C = error term
        self.C = C
        self.w = 0
        self.b = 0

    # Hinge Loss Function / Calculation
    def hingeloss(self, w, b, x, y):
        # Regularizer term
        reg = 0.5 * (w * w)

        for i in range(x.shape[0]):
            # Optimization term
            opt_term = y[i] * ((np.dot(w, x[i])) + b)

            # calculating loss
            loss = reg + self.C * max(0, 1-opt_term)
        return loss[0][0]

    def fit(self, X, Y, batch_size=100000, learning_rate=0.0001, epochs=1000):
        # The number of features in X
        number_of_features = X.shape[1]

        # The number of Samples in X
        number_of_samples = X.shape[0]

        c = self.C

        # Creating ids from 0 to number_of_samples - 1
        ids = np.arange(number_of_samples)

        # Shuffling the samples randomly
        np.random.shuffle(ids)

        # creating an array of zeros
        w = np.zeros((1, number_of_features))
        b = 0
        losses = []

        # Gradient Descent logic
        for i in range(epochs):
            # Calculating the Hinge Loss
            l = self.hingeloss(w, b, X, Y)

            # Appending all losses 
            losses.append(l)
            
            # Starting from 0 to the number of samples with batch_size as interval
            for batch_initial in range(0, number_of_samples, batch_size):
                gradw = 0
                gradb = 0

                for j in range(batch_initial, batch_initial+ batch_size):
                    if j < number_of_samples:
                        x = ids[j]
                        ti = Y[x] * (np.dot(w, X[x].T) + b)

                        if ti > 1:
                            gradw += 0
                            gradb += 0
                        else:
                            # Calculating the gradients

                            #w.r.t w 
                            gradw += c * Y[x] * X[x]
                            # w.r.t b
                            gradb += c * Y[x]

                # Updating weights and bias
                w = w - learning_rate * w + learning_rate * gradw
                b = b + learning_rate * gradb
        
        self.w = w
        self.b = b

        return self.w, self.b, losses

    def predict(self, X):
        prediction = np.dot(X, self.w[0]) + self.b # w.x + b
        return np.sign(prediction)

In [2]:
def float_to_int_delta(value, byte):

    import copy

    max_value = 50
    v = copy.deepcopy(value)
    delta = max_value/(pow(2,8*byte)-1)
    converted_int = int(v/delta)

    if(converted_int >= (pow(2,8*byte)/2)-1): 
        return int(pow(2,8*byte)/2)-1
    
    elif(converted_int <= -(pow(2,8*byte)/2)+1): 
        return -int(pow(2,8*byte)/2)+1
    
    else: 
        return converted_int

def int_to_float_delta(value, byte): 
    import copy
    max_value = 50
    v = copy.deepcopy(value)
    delta = max_value/(pow(2,8*byte)-1)
    return v*delta

In [3]:
int_to_float_delta(float_to_int_delta(24,2),2)

23.999389639124132

In [4]:
def int_to_svm_model_name(value): 
    first_value = int((value - value%16)/16)
    second_value = value%16
    return str(first_value), str(second_value) 

In [5]:
def string_int_to_hex_int(str_hex): 
    return 16*int(str_hex[0]) + int(str_hex[1]) 

In [6]:
def linear_train_int_bin(file_path, class_1, class_2, b, w): 
        #svm_model format: [class1, class2, b, w_i[]]

    #Modus: type of svm model. Linear, rbf, sparse etc.

    import struct 
    
    svm_model_name = ""

    c1 = str(class_1) 
    c2 = str(class_2)

    if(len(c1) == 1): 
        c1 = "0" + c1

    if(len(c2) == 1): 
        c2 = "0" + c2

    svm_model_name = "lsm" + c1 + c2

    file = open(file_path + svm_model_name, "wb")    
    
    buf = struct.pack("BB", class_1, class_2)

    file.write(buf)

    buf = struct.pack('h', b)

    file.write(buf)

    w_size = str(len(w)) + "h"

    buf = struct.pack(w_size, *w)
    
    file.write(buf)

    file.close()

    return 0

In [7]:
def bdt_config_bin(file_path, bdt_config_aray, config_num):

    import struct 

    bca = np.array(bdt_config_aray).flatten()

    bdt_config_name = "bdtc" + str(config_num)
    
    file = open(file_path + bdt_config_name, "wb")

    for i in range(len(bca)):
        buf = struct.pack(">B", bca[i]) 
        file.write(buf)

    file.close()

In [8]:
def hsi_binary(file_path, filename, X):

    pH = np.array(X).flatten()
        
    file = open(file_path + filename, "wb")

    for i in range(len(pH)):
        buf = struct.pack(">B", pH[i]) 
        file.write(buf)

    file.close()


In [9]:
def models_names(bdt, model_names): 
    
    svm_model_list = []

    for i in range(len(bdt)): 
        c1, c2 = int_to_svm_model_name(bdt[i][0])
        
        if(len(c1) == 1): 
            c1 = "0" + c1

        if(len(c2) == 1): 
            c2 = "0" + c2

        svm_model_list.append(str(model_names) + str(c1) + str(c2)) 

    return svm_model_list

In [10]:
def linear_svm_models_load(path, bdt): 

    import os 
    import struct

    svm_models_array = []

    svm_default_name = "lsm"

    svm_model_list = models_names(bdt, svm_default_name)

    for i in range(len(svm_model_list)): 
        file = open(path + svm_model_list[i], "rb")

        unpack_opt = "2b" + str(int(os.stat(path + svm_model_list[i]).st_size/2 - 1))  + "h"
        content = struct.unpack(unpack_opt,file.read())

        svm_model = []
        svm_model.append(content[0])
        svm_model.append(content[1])

        for j in range(2,len(content)): 
            svm_model.append(int_to_float_delta(content[j],2))
        
        svm_models_array.append(svm_model)

        file.close()

    return svm_models_array  

In [11]:
def rotate_image(image): 
    rotated_image = []
    for i in range(len(image[0])): 
        x_array = []
        for j in range(len(image)-1,-1,-1):
            x_array.append(image[j][i])
        rotated_image.append(x_array)

    return np.array(rotated_image)

In [12]:
def svm_branch_models_to_bin(svm_branch_model, b_extra):

    sbm = copy.deepcopy(svm_branch_model)
    class_1 = sbm.classes_[0]
    class_2 = sbm.classes_[1]
    b = float_to_int_delta(sbm.intercept_[0] + b_extra ,2)
    w = sbm.coef_[0]
    tw = []

    for i in range(len(w)): 
        tw.append(float_to_int_delta(w[i],2))

    linear_train_int_bin("svm_model_files/", class_1, class_2, b, tw)

In [13]:
def bdt_string_to_config_format(bdt):

    import copy

    config_bdt = []

    for i in range(len(bdt)): 
        bdt_index = []
        bdt_i = copy.deepcopy(bdt[i][0]) 

        if(bdt_i[2] == "0"): 
            bdt_i = bdt_i[0] + bdt_i[1] + bdt_i[3] 

        if(bdt_i[0] == "0"): 
            bdt_i = bdt_i[1] + bdt_i[2]

        bdt_index.append(string_int_to_hex_int(bdt_i))
        bdt_index.append(int(bdt[i][1]))
        bdt_index.append(int(bdt[i][2]))
        config_bdt.append(bdt_index)

    return config_bdt

bdt = bdt_string_to_config_format([["0203", "1", "2"]])

print(bdt)

#linear_svm_models_load("svm_model_files", bdt)

[[35, 1, 2]]


In [14]:
def svm_linear(X_i, svm_model):
    pred = 0 

    for i in range(len(X_i)): 
        pred += (X_i[i] * svm_model[3+i]) + svm_model[2]
        #pred = np.dot(X_i, np.array(svm_model[3:len(X_i)+3]).T) + svm_model[2]
    if(pred >= 0): 
        return -1
    else:
        return 1

In [15]:
def bdt_execute(X, bdt): 

    # bdt : [svm_model_classes, send_index1, send_index2]
    
    svm_models = linear_svm_models_load("svm_model_files/", bdt)
    svm_models_index = 0 
    prevoius_index = 0 
    pixel_holder = 0 
    labeled_image = [] 

    for i in range(len(X)): 

        svm_models_index = 0

        while(True):

            prevoius_index = svm_models_index

            pixel_holder = svm_linear(X[i], svm_models[svm_models_index])

            if(pixel_holder >= 0): 
                svm_models_index = int(bdt[svm_models_index][1])

            else: 
                svm_models_index = int(bdt[svm_models_index][2])

            if(svm_models_index == 0): 

                if(pixel_holder >= 0): 
                    labeled_image.append(svm_models[prevoius_index][0])
                else: 
                    labeled_image.append(svm_models[prevoius_index][1])

                break

    return labeled_image, svm_models

In [85]:
from SVMBDT import *
from HelperFunctions_ import *
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

trondheim_data = plt.imread("../Training_data/Trondheim_2022_08_23T10_26_43-bin3.png")
trondheim_gt = np.load("../Training_data/Trondheim_labels_2022-08-23.npz")["labels"]

#Preprocess the labeling 

new_trondheim_gt = []
y = trondheim_gt.flatten()

for i in range(0,len(y),3): 
    new_trondheim_gt.append(y[i])

new_trondheim_gt = np.array(new_trondheim_gt)

#Make labels into land, water and clouds 
#water  = 2
#cropland = 3
#pasture = 4
#city = 5
#rock = 6
#thin_forest = 7
#thick_forest = 8 
#cloud = 9
#thin_clouds = 10

trondheim_new_labels = copy.deepcopy(new_trondheim_gt)


trondheim_new_labels[trondheim_new_labels == 10] = 9
#trondheim_new_labels[trondheim_new_labels == 9] = 3

nl = [4,5,6,7,8]

for i in range(len(nl)):
        trondheim_new_labels[trondheim_new_labels == nl[i]] = 3

print(classesInLabels(trondheim_new_labels))

new_trondheim = rotate_image(trondheim_data)

X = np.array(new_trondheim).reshape((-1,3))
#X = preprocessing.scale(X, axis=0)
y = trondheim_new_labels.flatten()

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.9, random_state=1)


tb = {0 : [[3,9],[2]], 1 : [[[3],[9]],[]]}
svm_tb = {0 : LinearSVC(C = 1, class_weight="balanced", intercept_scaling=1), 1 : [LinearSVC(C = 1, class_weight="balanced", intercept_scaling=1), False]}
#svm_tb = {0 : SVM(), 1 : [SVM(), False]}

svm_branch_models = SvmDesionTreeTrain(X_train, y_train, tb, svm_tb)

[2 3 9]
The training time is: 0.074 sec.


In [121]:
svm_branch_models_to_bin(svm_branch_models[0],0)
svm_branch_models_to_bin(svm_branch_models[1][0],1.6)


In [122]:
s_bdt = [["0203", "0", "1"],["0309", "0", "0"]]
#s_bdt = [["0203", "0", "0"]]
bdt = bdt_string_to_config_format(s_bdt)

pl, svm_models = bdt_execute(X, bdt)

In [19]:
SvmDesionTreePredict(X, svm_branch_models, tb)

The first branch: 0.01
The prediction time is: 0.481 sec.


array([9, 9, 9, ..., 3, 3, 3], dtype=int16)

In [123]:
pl = np.array(pl)
plt.imshow(pl.reshape(956,228))
plt.axis('off')
plt.savefig("Tronhdeim_few_labels", dpi = 300, bbox_inches="tight")
plt.close()

In [124]:
classesInLabels(pl)

array([2, 3, 9])

In [110]:
pl[len(pl)-1]

3

In [23]:
import scipy.io as sc
gt = sc.loadmat("../Training_data/end3.mat")
samson = sc.loadmat("../Training_data/samson.mat")['V']
labels = np.argmax(gt['A'].T.reshape((-1,3)), axis=1)

new_labels = copy.deepcopy(labels)
#new_labels[new_labels == 1] = 0  
samson = reshape_sj(copy.deepcopy(samson))

X = samson.transpose(0,2,1).reshape((-1,156))
#X = preprocessing.scale(X, axis=0)
y = new_labels.flatten()

# split samson into train test sets
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.5, random_state=1)

tb = {0 : [[0,1],[2]], 1 : [[[0],[1]],[]]}
svm_tb = {0 : LinearSVC(C = 1, class_weight="balanced", intercept_scaling=1), 1 : [LinearSVC(C = 1, class_weight="balanced", intercept_scaling=1), False]}
#svm_tb = {0 : SVM(), 1 : [SVM(), False]}

svm_branch_models = SvmDesionTreeTrain(X_train, y_train, tb, svm_tb)

The training time is: 0.101 sec.


In [24]:
np.min(X)

0.0

In [79]:
svm_branch_models_to_bin(svm_branch_models[0],-2.373)
svm_branch_models_to_bin(svm_branch_models[1][0],-0.52)

In [80]:
def bdt_execute(X, bdt): 

    # bdt : [svm_model_classes, send_index1, send_index2]
    
    svm_models = linear_svm_models_load("svm_model_files/", bdt)
    svm_models_index = 0 
    prevoius_index = 0 
    pixel_holder = 0 
    labeled_image = [] 

    for i in range(len(X)): 

        svm_models_index = 0

        while(True):

            prevoius_index = svm_models_index

            pixel_holder = svm_linear(X[i], svm_models[svm_models_index])

            if(pixel_holder >= 0): 
                svm_models_index = int(bdt[svm_models_index][1])

            else: 
                svm_models_index = int(bdt[svm_models_index][2])

            if(svm_models_index == 0): 

                if(pixel_holder >= 0): 
                    labeled_image.append(svm_models[prevoius_index][0])
                else: 
                    labeled_image.append(svm_models[prevoius_index][1])

                break

    return labeled_image, svm_models

In [81]:
#s_bdt = [["0203", "1", "0"],["0209", "0", "0"]]
s_bdt = [["0002", "1", "0"], ["0001", "0", "0"]]
bdt = bdt_string_to_config_format(s_bdt)
pl, svm_models = bdt_execute(X, bdt)

In [82]:
pl = np.array(pl)
plt.imshow(pl.reshape(95,95))
plt.axis('off')
plt.savefig("Samson_few_labels2", dpi = 300, bbox_inches="tight")
plt.close()

In [83]:
classesInLabels(pl)

array([0, 1, 2])

In [125]:
np.sum(pl == y)/len(y)

0.883271856419291